In [ ]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import search_history
from utils import make_recommendations
from utils import get_recommended_homes


user_id=666
user_type=1

user_actions = search_history(user_id, user_type)
homes = pd.read_csv('data/homes_fixed_scaled.csv')

recs = make_recommendations(homes, user_actions)
df_recommendations = get_recommended_homes(recs, user_id)

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



In [1]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import rec_system

user_id=666
user_type=1

test = rec_system(1)
test

utils.py загр


/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



,id,price,level,rooms,area,kitchen_area,geo_lat,geo_lon
0,588,5516100,5,-1,24.30,0.0,55.754047,37.620405
1,675,5658000,4,-1,28.29,0.0,55.668127,37.737237
2,684,5658000,4,-1,28.29,0.0,55.668127,37.737237
3,88,5541998,10,-1,24.00,0.0,55.575745,37.435046
4,593,5741767,9,-1,24.07,0.0,55.754047,37.620405
5,789,5421314,4,-1,24.40,0.0,55.742435,37.707553
6,685,5658000,10,-1,28.29,0.0,55.668127,37.737237
7,61,5791200,6,-1,25.40,0.0,55.516860,37.349500
8,567,5536250,4,-1,21.50,0.0,55.754047,37.620405
9,835,5616300,3,-1,29.10,0.0,55.891209,37.577463


In [11]:
test = rec_system(666, 1)
test

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



,id,price,level,rooms,area,kitchen_area,geo_lat,geo_lon
0,19,6884515,6,-1,30.50,0.0,55.449414,36.999877
1,24,7043381,9,-1,28.30,0.0,55.449414,36.999877
2,178,7118833,5,-1,30.60,0.0,55.567659,37.484608
3,808,6923977,10,-1,27.50,0.0,55.707644,37.749274
4,1707,7000000,3,1,27.63,0.0,55.698767,37.462159
5,2005,7200000,6,1,34.00,0.0,55.600624,37.660620
6,722,7147902,7,-1,25.51,0.0,55.886918,37.534136
7,909,6900000,3,-1,27.00,0.0,55.773763,37.739716
8,1534,7141733,12,1,28.30,0.0,55.564963,37.498512
9,934,7185923,7,-1,24.61,0.0,55.792997,37.740690


In [9]:
df_recommendations

,id,price,level,rooms,area,kitchen_area,geo_lat,geo_lon
0,615,6655460,8,-1,27.40,0.0,55.881954,37.501923
1,3623,6686836,7,1,25.09,0.0,55.792997,37.740690
2,66,6500000,8,-1,24.80,0.0,55.521764,37.354620
3,234,6500000,10,-1,27.00,0.0,55.967260,37.153689
4,3621,6558909,7,1,24.61,0.0,55.792997,37.740690
5,57,6630000,3,-1,28.00,0.0,55.548706,37.311112
6,595,6411876,10,-1,27.77,0.0,55.754047,37.620405
7,183,6337495,8,-1,28.50,0.0,55.567659,37.484608
8,1208,6500000,10,1,23.60,0.0,55.623345,37.304879
9,19,6884515,6,-1,30.50,0.0,55.449414,36.999877


In [18]:
final_recommendations = recs
homes_fixed = homes.copy()

In [19]:
# Создадим структуру для хранения результатов тестов
stat_test_results = {}

# Выполнение t-теста для каждого пользователя
for user in final_recommendations.keys():
    interacted_obj_ids = user_actions[user_actions['user_id'] == user]['obj_id']
    recommended_obj_ids = final_recommendations[user]

    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]
    recommended_objects = homes_fixed.loc[homes_fixed.index.intersection(recommended_obj_ids)]

    if not interacted_objects.empty and not recommended_objects.empty:
        # Средние значения по объектам
        interacted_mean = interacted_objects.mean(axis=0)
        recommended_mean = recommended_objects.mean(axis=0)

        # Проведение t-теста
        t_stat, p_val = ttest_ind(interacted_mean, recommended_mean, equal_var=False, nan_policy='omit')

        # Сохранение результатов
        stat_test_results[user] = {'t_stat': t_stat, 'p_val': p_val}

        # Вывод результатов для каждого пользователя
        print(f'User {user}: t_stat = {t_stat:.5f}, p_val = {p_val:.5f}')

# Преобразуем в DataFrame для удобного анализа
stat_test_results_df = pd.DataFrame.from_dict(stat_test_results, orient='index')

# Подсчет доли пользователей, у которых p_val > 0.05
no_significant_diff_ratio = (stat_test_results_df['p_val'] > 0.05).mean()
print(f'Доля пользователей, у которых нет статистически значимой разницы: {no_significant_diff_ratio:.2%}')


User 666: t_stat = -0.00778, p_val = 0.99390
Доля пользователей, у которых нет статистически значимой разницы: 100.00%


если p_val > 0.05, то нет оснований полагать, что есть стат. значимая разница => рекомендации были хорошими для всех юзеров